# Predicting Clicks on Job Listings

#### Project: We were given a time series dataset of snapshot of the state of each job listed on every day from December 2016 to January 2018 and no instructions. The dataset contained over 4 million rows.

#### Goal: Predict whether the job listing will get 1000 or more total clicks over the life of it's posting on Indeed.com using Deep Learning

## Step 1 - Data Wrangling & Feature Engineering

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Import csv file into a Pandas Dataframe

In [2]:
df_virgin = pd.read_csv('datafest.csv')
df_virgin.head()

,employerJobCount,date,country,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,...,clicks,yearWeekOfYear,advDateCreatedDate,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash
0,1,2016-12-02,US,Company360777264968,20160825,none,0,99,20160825,650,...,27,201648,2016-08-25,150-249,optometrist,Retail/Consumer Products,meddr,OH,Milan,Company3607772649681
1,35,2016-12-01,US,Company489626272656,20120522,Direct Employer,0,98,20160825,4794,...,11,201648,2012-05-22,1000+,virtualization engineer,Accounting/Consulting,techsoftware,DC,Washington,Company48962627265635
2,10,2016-12-03,US,Company558345748547,20151112,Direct Employer,0,99,20160826,366,...,35,201648,2015-11-12,1000+,scada engineer,Food/Beverage Services,techsoftware,CA,American Canyon,Company55834574854710
3,1,2016-12-01,US,Company1391569415594,20160826,none,0,97,20160826,482,...,39,201648,2016-08-26,1000+,journeyman carpenter,Real Estate,construction,CT,New Haven,Company13915694155941
4,126,2016-12-03,US,Company1700807049289,20110325,Direct Employer,0,99,20160826,3427,...,31,201648,2011-03-25,1000+,front end manager,Non-Profit/Association,retail,LA,Algiers,Company1700807049289126


In [3]:
# Create a fresh DataFrame from the untouched DataFrame
df = df_virgin

#### Change Type of Date Columns to DateTime Objects

In [4]:
df.dtypes

employerJobCount                       int64
date                                  object
country                               object
maskedEmployerName                    object
employerAccountDateCreated             int64
advtype                               object
avgoverallcompanyrating                int64
jobagedays                             int64
jobdatecreated                         int64
descriptionLengthChars                 int64
candStatusReviewedCount                int64
supervisingJob                         int64
licenseRequiredJob                     int64
noEducationRequirementsJob             int64
highSchoolEducationRequirementsJob     int64
higherEducationRequirementsJob         int64
applies                                int64
clicks                                 int64
yearWeekOfYear                         int64
advDateCreatedDate                    object
employeecount                         object
normtitle                             object
employerIn

In [5]:
df['date'] = pd.to_datetime(df['date'])
# df['employerAccountDateCreated'] = pd.to_datetime(df['employerAccountDateCreated'])
# df['jobdatecreated'] = pd.to_datetime(df['jobdatecreated'])
df['advDateCreatedDate'] = pd.to_datetime(df['advDateCreatedDate'])

In [6]:
df.dtypes

employerJobCount                               int64
date                                  datetime64[ns]
country                                       object
maskedEmployerName                            object
employerAccountDateCreated                     int64
advtype                                       object
avgoverallcompanyrating                        int64
jobagedays                                     int64
jobdatecreated                                 int64
descriptionLengthChars                         int64
candStatusReviewedCount                        int64
supervisingJob                                 int64
licenseRequiredJob                             int64
noEducationRequirementsJob                     int64
highSchoolEducationRequirementsJob             int64
higherEducationRequirementsJob                 int64
applies                                        int64
clicks                                         int64
yearWeekOfYear                                

#### Sort the DataFrame By Date in Descending Order

In [7]:
df_sorted = df.sort_values(by='date')

In [8]:
df_sorted.head(5)

,employerJobCount,date,country,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,...,clicks,yearWeekOfYear,advDateCreatedDate,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash
1147982,329,2016-12-01,US,Company377957122300,20151204,Staffing Agency,0,9,20161122,872,...,23,201648,2015-12-04,750-999,pro services assistant,Education,customer,AL,Baileyton,Company377957122300329
1182201,15,2016-12-01,US,Company1529008363619,20150407,Direct Employer,0,33,20161029,205,...,39,201648,2015-04-07,1-49,fitness trainer,Retail/Consumer Products,sports,CT,Berlin,Company152900836361915
1975400,9,2016-12-01,US,Company1288490194881,20160511,Staffing Agency,0,80,20160912,1041,...,51,201648,2016-05-11,50-149,cleaner,Telecom,sanitation,IN,Evanston,Company12884901948819
1182252,24,2016-12-01,US,Company1357209667334,20160725,Staffing Agency,29,31,20161031,2317,...,43,201648,2016-07-25,50-149,junior compliance officer,Non-Profit/Association,legal,FL,Tarpon Springs Sponge Docks,Company135720966733424
1975386,82,2016-12-01,US,Company120259084463,20150730,Staffing Agency,0,83,20160909,1239,...,43,201648,2015-07-30,50-149,test engineer,Staffing Firm (General),engid,GA,Morganton,Company12025908446382


In [9]:
df_sorted.tail(5)

,employerJobCount,date,country,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,...,clicks,yearWeekOfYear,advDateCreatedDate,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash
2859877,795,2018-01-31,US,Company730144440322,20120514,Staffing Agency,41,90,20171102,971,...,27,20185,2012-05-14,50-149,family medicine physician,Staffing Firm (General),meddr,IA,Ackworth,Company730144440322795
2657503,111,2018-01-31,US,Company661424967104,20151002,Direct Employer,50,48,20171214,3509,...,27,20185,2015-10-02,1-49,senior cook,Health Care/Medical,food,KY,Lexington,Company661424967104111
2502368,3,2018-01-31,US,Company575525632080,20141014,Direct Employer,0,62,20171130,1917,...,35,20185,2014-10-14,750-999,logistics clerk,Food/Beverage Services,admin,NY,Canandaigua,Company5755256320803
3289430,8,2018-01-31,US,Company111669175609,20180115,none,0,0,20180131,298,...,23,20185,2018-01-15,1-49,controller,Automotive,accounting,AR,Searcy,Company1116691756098
2687493,183,2018-01-31,US,Company824633722102,20140319,Direct Employer,0,21,20180110,1550,...,19,20185,2014-03-19,250-499,care specialist,Non-Profit/Association,care,PA,Honesdale,Company824633722102183


#### Feature Engineering totalClicks and totalApplies

In [10]:
df_features = df_sorted.groupby(['jobHash'])['clicks','applies'].sum().reset_index()
df_features = df_features.rename(columns={'clicks': 'totalClicks', 'applies': 'totalApplies'})
df_features.head()

,jobHash,totalClicks,totalApplies
0,Company100032,291,59
1,Company10005,637,107
2,Company100104,782,124
3,Company10048,116,18
4,Company10050223472701197,132,22


#### Using 'jobHash' column as the primary key, drop duplicates and keep last entry

In [11]:
df_unique = df_sorted.drop_duplicates(subset='jobHash', keep='last', inplace=False)
df_unique.head()

,employerJobCount,date,country,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,...,clicks,yearWeekOfYear,advDateCreatedDate,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash
1182890,305,2016-12-01,US,Company1288490188873,20160211,Direct Employer,0,16,20161115,2116,...,27,201648,2016-02-11,150-249,solar installer,Technology/Information,install,DC,Washington,Company1288490188873305
1192844,2,2016-12-01,US,Company292057776172,20160817,Staffing Agency,0,94,20160829,608,...,27,201648,2016-08-17,1-49,unit leader,Arts/Entertainment/Recreation,retail,NJ,Plainfield,Company2920577761722
1192836,4,2016-12-01,US,Company154618829386,20130308,Direct Employer,30,98,20160825,591,...,39,201648,2013-03-08,1-49,registered nurse - \tcardiac electrophysiology...,Education,mednurse,IA,Luther,Company1546188293864
1192835,5,2016-12-01,US,Company1709396994639,20130925,Direct Employer,0,99,20160824,740,...,27,201648,2013-09-25,1000+,process engineer,Financial Services/Banking,engid,NY,Uniondale,Company17093969946395
397047,1,2016-12-01,US,Company1494648623682,20160824,none,0,99,20160824,430,...,39,201648,2016-08-24,1-49,industrial designer,Legal,arts,FL,Clearwater,Company14946486236821


#### Merge new Features

In [12]:
dfx = pd.merge(df_unique, df_features, on='jobHash')
dfx.head()

,employerJobCount,date,country,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,...,advDateCreatedDate,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash,totalClicks,totalApplies
0,305,2016-12-01,US,Company1288490188873,20160211,Direct Employer,0,16,20161115,2116,...,2016-02-11,150-249,solar installer,Technology/Information,install,DC,Washington,Company1288490188873305,27,4
1,2,2016-12-01,US,Company292057776172,20160817,Staffing Agency,0,94,20160829,608,...,2016-08-17,1-49,unit leader,Arts/Entertainment/Recreation,retail,NJ,Plainfield,Company2920577761722,27,4
2,4,2016-12-01,US,Company154618829386,20130308,Direct Employer,30,98,20160825,591,...,2013-03-08,1-49,registered nurse - \tcardiac electrophysiology...,Education,mednurse,IA,Luther,Company1546188293864,39,7
3,5,2016-12-01,US,Company1709396994639,20130925,Direct Employer,0,99,20160824,740,...,2013-09-25,1000+,process engineer,Financial Services/Banking,engid,NY,Uniondale,Company17093969946395,27,4
4,1,2016-12-01,US,Company1494648623682,20160824,none,0,99,20160824,430,...,2016-08-24,1-49,industrial designer,Legal,arts,FL,Clearwater,Company14946486236821,39,7


In [13]:
dfx.dtypes

employerJobCount                               int64
date                                  datetime64[ns]
country                                       object
maskedEmployerName                            object
employerAccountDateCreated                     int64
advtype                                       object
avgoverallcompanyrating                        int64
jobagedays                                     int64
jobdatecreated                                 int64
descriptionLengthChars                         int64
candStatusReviewedCount                        int64
supervisingJob                                 int64
licenseRequiredJob                             int64
noEducationRequirementsJob                     int64
highSchoolEducationRequirementsJob             int64
higherEducationRequirementsJob                 int64
applies                                        int64
clicks                                         int64
yearWeekOfYear                                

In [14]:
dfx = dfx.drop(['country','applies','clicks'], axis=1)

In [15]:
dfx.head()

,employerJobCount,date,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,candStatusReviewedCount,...,advDateCreatedDate,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash,totalClicks,totalApplies
0,305,2016-12-01,Company1288490188873,20160211,Direct Employer,0,16,20161115,2116,2,...,2016-02-11,150-249,solar installer,Technology/Information,install,DC,Washington,Company1288490188873305,27,4
1,2,2016-12-01,Company292057776172,20160817,Staffing Agency,0,94,20160829,608,2,...,2016-08-17,1-49,unit leader,Arts/Entertainment/Recreation,retail,NJ,Plainfield,Company2920577761722,27,4
2,4,2016-12-01,Company154618829386,20130308,Direct Employer,30,98,20160825,591,6,...,2013-03-08,1-49,registered nurse - \tcardiac electrophysiology...,Education,mednurse,IA,Luther,Company1546188293864,39,7
3,5,2016-12-01,Company1709396994639,20130925,Direct Employer,0,99,20160824,740,3,...,2013-09-25,1000+,process engineer,Financial Services/Banking,engid,NY,Uniondale,Company17093969946395,27,4
4,1,2016-12-01,Company1494648623682,20160824,none,0,99,20160824,430,1,...,2016-08-24,1-49,industrial designer,Legal,arts,FL,Clearwater,Company14946486236821,39,7


In [16]:
dfx.loc[dfx['maskedEmployerName'] == 'Company1288490188873']

,employerJobCount,date,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,candStatusReviewedCount,...,advDateCreatedDate,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash,totalClicks,totalApplies
0,305,2016-12-01,Company1288490188873,20160211,Direct Employer,0,16,20161115,2116,2,...,2016-02-11,150-249,solar installer,Technology/Information,install,DC,Washington,Company1288490188873305,27,4
90,410,2016-12-01,Company1288490188873,20160211,Direct Employer,0,0,20161201,3073,6,...,2016-02-11,1-49,senior special events manager,Technology/Information,management,MD,Columbia,Company1288490188873410,43,8
799,431,2016-12-03,Company1288490188873,20160211,Direct Employer,0,0,20161203,1882,1,...,2016-02-11,1-49,receptionist/clerk,Technology/Information,admin,FL,ChampionsGate,Company1288490188873431,19,2
6461,469,2016-12-19,Company1288490188873,20160211,Direct Employer,0,12,20161207,1239,1,...,2016-02-11,1-49,junior support engineer,Technology/Information,techhelp,IL,Farina,Company1288490188873469,174,27
23593,725,2017-02-04,Company1288490188873,20160211,Direct Employer,0,1,20170203,2116,0,...,2016-02-11,1-49,financial analyst,Technology/Information,accounting,VA,Arlington,Company1288490188873725,50,7
25632,756,2017-02-08,Company1288490188873,20160211,Direct Employer,0,1,20170207,1863,0,...,2016-02-11,1-49,automotive mechanic,Technology/Information,install,TX,San Antonio,Company1288490188873756,86,16
31944,902,2017-02-25,Company1288490188873,20160211,Direct Employer,0,3,20170222,1855,1,...,2016-02-11,1-49,senior engagement manager,Technology/Information,management,NY,Mount Kisco,Company1288490188873902,31,5
33579,893,2017-02-28,Company1288490188873,20160211,Direct Employer,0,6,20170222,1912,0,...,2016-02-11,1-49,home care aide,Technology/Information,care,MD,Silver Spring,Company1288490188873893,69,9
70906,1030,2017-05-21,Company1288490188873,20160211,Direct Employer,0,9,20170512,2156,0,...,2016-02-11,1-49,records coordinator,Technology/Information,admin,OH,Columbus,Company12884901888731030,170,26
82390,1036,2017-06-12,Company1288490188873,20160211,Direct Employer,0,27,20170516,2320,1,...,2016-02-11,1-49,operations intern,Technology/Information,admin,DC,Washington,Company12884901888731036,205,32


In [17]:
len(dfx.columns)

26

In [18]:
dfx['maskedEmployerName'].value_counts().head(10)

Company695784701963     251
Company1022202216469    229
Company695784702007     212
Company919123001450     196
Company712964571203     194
Company1022202216485    165
Company1486058684475    153
Company249108103176     142
Company1288490188876    129
Company850403524675     120
Name: maskedEmployerName, dtype: int64

In [19]:
df_emu = dfx.groupby(['maskedEmployerName']).size().reset_index()

In [20]:
df_emu = df_emu.rename(columns={0: 'jobsListedByEmp'})
df_emu.head()

,maskedEmployerName,jobsListedByEmp
0,Company1000,2
1,Company10010,1
2,Company1004,1
3,Company1005022347270,9
4,Company1005022347271,22


In [21]:
dfx = pd.merge(dfx, df_emu, on='maskedEmployerName')
dfx.head()

,employerJobCount,date,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,candStatusReviewedCount,...,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash,totalClicks,totalApplies,jobsListedByEmp
0,305,2016-12-01,Company1288490188873,20160211,Direct Employer,0,16,20161115,2116,2,...,150-249,solar installer,Technology/Information,install,DC,Washington,Company1288490188873305,27,4,11
1,410,2016-12-01,Company1288490188873,20160211,Direct Employer,0,0,20161201,3073,6,...,1-49,senior special events manager,Technology/Information,management,MD,Columbia,Company1288490188873410,43,8,11
2,431,2016-12-03,Company1288490188873,20160211,Direct Employer,0,0,20161203,1882,1,...,1-49,receptionist/clerk,Technology/Information,admin,FL,ChampionsGate,Company1288490188873431,19,2,11
3,469,2016-12-19,Company1288490188873,20160211,Direct Employer,0,12,20161207,1239,1,...,1-49,junior support engineer,Technology/Information,techhelp,IL,Farina,Company1288490188873469,174,27,11
4,725,2017-02-04,Company1288490188873,20160211,Direct Employer,0,1,20170203,2116,0,...,1-49,financial analyst,Technology/Information,accounting,VA,Arlington,Company1288490188873725,50,7,11


In [22]:
dfx = dfx.drop(['employerJobCount'], axis=1)

In [23]:
dfx.head()

,date,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,candStatusReviewedCount,supervisingJob,...,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash,totalClicks,totalApplies,jobsListedByEmp
0,2016-12-01,Company1288490188873,20160211,Direct Employer,0,16,20161115,2116,2,0,...,150-249,solar installer,Technology/Information,install,DC,Washington,Company1288490188873305,27,4,11
1,2016-12-01,Company1288490188873,20160211,Direct Employer,0,0,20161201,3073,6,0,...,1-49,senior special events manager,Technology/Information,management,MD,Columbia,Company1288490188873410,43,8,11
2,2016-12-03,Company1288490188873,20160211,Direct Employer,0,0,20161203,1882,1,0,...,1-49,receptionist/clerk,Technology/Information,admin,FL,ChampionsGate,Company1288490188873431,19,2,11
3,2016-12-19,Company1288490188873,20160211,Direct Employer,0,12,20161207,1239,1,0,...,1-49,junior support engineer,Technology/Information,techhelp,IL,Farina,Company1288490188873469,174,27,11
4,2017-02-04,Company1288490188873,20160211,Direct Employer,0,1,20170203,2116,0,0,...,1-49,financial analyst,Technology/Information,accounting,VA,Arlington,Company1288490188873725,50,7,11


### Drop Rows containing Null Values

In [24]:
dfx = dfx.dropna(axis=0, how='any')
len(dfx)

235361

#### Save new DataFrame to CSV

In [25]:
dfx.to_csv('datafest_cleanv2.csv', index=False)

In [35]:
dfx.head()

,date,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,candStatusReviewedCount,supervisingJob,...,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash,totalClicks,totalApplies,jobsListedByEmp
0,2016-12-01,Company1288490188873,20160211,Direct Employer,0,16,20161115,2116,2,0,...,150-249,solar installer,Technology/Information,install,DC,Washington,Company1288490188873305,27,4,11
1,2016-12-01,Company1288490188873,20160211,Direct Employer,0,0,20161201,3073,6,0,...,1-49,senior special events manager,Technology/Information,management,MD,Columbia,Company1288490188873410,43,8,11
2,2016-12-03,Company1288490188873,20160211,Direct Employer,0,0,20161203,1882,1,0,...,1-49,receptionist/clerk,Technology/Information,admin,FL,ChampionsGate,Company1288490188873431,19,2,11
3,2016-12-19,Company1288490188873,20160211,Direct Employer,0,12,20161207,1239,1,0,...,1-49,junior support engineer,Technology/Information,techhelp,IL,Farina,Company1288490188873469,174,27,11
4,2017-02-04,Company1288490188873,20160211,Direct Employer,0,1,20170203,2116,0,0,...,1-49,financial analyst,Technology/Information,accounting,VA,Arlington,Company1288490188873725,50,7,11


### Features engineered:

Number of Days listed - check

Total clicks of Job post - check

Total Applies - check

Jobs Posted By Employer - check

# Step 2 - Data Exploration

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('datafest_clean.csv')

In [3]:
df.head()

,date,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,candStatusReviewedCount,supervisingJob,...,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash,totalClicks,totalApplies,jobsListedByEmp
0,2016-12-01,Company1288490188873,20160211,Direct Employer,0,16,20161115,2116,2,0,...,150-249,solar installer,Technology/Information,install,DC,Washington,Company1288490188873305,27,4,11
1,2016-12-01,Company1288490188873,20160211,Direct Employer,0,0,20161201,3073,6,0,...,1-49,senior special events manager,Technology/Information,management,MD,Columbia,Company1288490188873410,43,8,11
2,2016-12-03,Company1288490188873,20160211,Direct Employer,0,0,20161203,1882,1,0,...,1-49,receptionist/clerk,Technology/Information,admin,FL,ChampionsGate,Company1288490188873431,19,2,11
3,2016-12-19,Company1288490188873,20160211,Direct Employer,0,12,20161207,1239,1,0,...,1-49,junior support engineer,Technology/Information,techhelp,IL,Farina,Company1288490188873469,174,27,11
4,2017-02-04,Company1288490188873,20160211,Direct Employer,0,1,20170203,2116,0,0,...,1-49,financial analyst,Technology/Information,accounting,VA,Arlington,Company1288490188873725,50,7,11


#### Total Unique Job Postings by Industry in the US

In [4]:
df['employerIndustry'].value_counts()

Staffing Firm (General)          26148
Health Care/Medical              24404
Other                            11526
Retail/Consumer Products         10773
Food/Beverage Services           10601
Technology/Information            9078
Construction/Engineering          8646
Transportation/Logistics          8564
Hospitality/Travel                8479
Education                         7776
Automotive                        7593
Non-Profit/Association            6950
Energy/Utilities                  6902
Manufacturing                     6869
Financial Services/Banking        6615
Real Estate                       6416
Accounting/Consulting             6339
Beauty/Wellness                   6239
Arts/Entertainment/Recreation     6205
Insurance                         5709
Defense/Security                  5625
Advertising/Marketing             5377
Agriculture                       5266
Legal                             5026
Government/Military               4900
Telecom                  

#### Most Frequent Job Title Postings in US

In [5]:
df['normtitle'].value_counts().head(25)

truck driver                       865
customer service representative    816
retail sales associate             625
sales representative               549
registered nurse                   532
nursing assistant                  518
administrative assistant           467
licensed practical nurse           437
delivery driver                    421
maintenance technician             367
warehouse worker                   354
receptionist                       336
laborer                            329
preschool teacher                  298
front desk agent                   285
cleaner                            284
physical therapist                 276
outside sales representative       274
forklift operator                  261
medical assistant                  259
cook                               256
home health aide                   256
restaurant manager                 247
housekeeper                        247
server                             246
Name: normtitle, dtype: i

#### Top 10 Cities with Most Tech Jobs

In [6]:
df_tech = df.loc[df['employerIndustry'] == 'Technology/Information']
df_tech['city'].value_counts().head(20)

New York         106
Washington        81
Chicago           77
Houston           69
Los Angeles       67
Austin            56
Atlanta           53
San Francisco     51
Dallas            47
Philadelphia      40
Columbus          40
Denver            39
San Diego         34
Boston            34
Seattle           32
Phoenix           32
Columbia          31
Miami             29
Orlando           29
Plano             28
Name: city, dtype: int64

#### Most Popular Industries in Arizona

In [7]:
df_az = df.loc[df['admin1'] == 'AZ']
df_az['employerIndustry'].value_counts()

Staffing Firm (General)          401
Health Care/Medical              386
Other                            180
Retail/Consumer Products         168
Food/Beverage Services           148
Hospitality/Travel               139
Technology/Information           135
Non-Profit/Association           132
Automotive                       124
Construction/Engineering         120
Education                        116
Financial Services/Banking       115
Beauty/Wellness                  111
Manufacturing                    106
Real Estate                      100
Accounting/Consulting             99
Transportation/Logistics          98
Energy/Utilities                  94
Defense/Security                  91
Arts/Entertainment/Recreation     87
Insurance                         83
Advertising/Marketing             83
Telecom                           75
Legal                             74
Media/Newspaper                   72
Work at Home                      72
Agriculture                       71
G

#### Top Posted Jobs in Arizona

In [8]:
df_az['normtitle'].value_counts().head(20)

customer service representative    19
administrative assistant           14
maintenance technician             13
truck driver                        9
sales representative                9
delivery driver                     8
x-ray technician                    8
cook                                8
nursing assistant                   8
server                              7
equipment operator                  7
registered nurse                    7
front desk agent                    6
caregiver                           6
laborer                             5
janitor                             5
hair stylist                        5
licensed practical nurse            5
house cleaner                       5
security guard                      5
Name: normtitle, dtype: int64

#### Most Competitive Companies

In [9]:
df_emp = df.groupby(['maskedEmployerName','jobsListedByEmp'])['totalClicks','totalApplies'].sum().reset_index()
df_emp['competi'] = df_emp.apply(lambda row: row.totalApplies / row.jobsListedByEmp, axis=1)
df_emp = df_emp.sort_values(by='competi')
df_emp.tail(20)

,maskedEmployerName,jobsListedByEmp,totalClicks,totalApplies,competi
14600,Company1159641175047,1,9333,2171,2171.0
5402,Company1056561956694,1,9358,2191,2191.0
84480,Company386547059355,1,8964,2219,2219.0
49309,Company1546188240869,1,9601,2326,2326.0
693,Company1005022372152,1,10065,2332,2332.0
99302,Company549755815207,1,9615,2390,2390.0
118110,Company755914246618,1,10350,2439,2439.0
120200,Company77309413895,1,10298,2525,2525.0
70061,Company231928235172,1,11113,2660,2660.0
69534,Company223338303804,1,11422,2696,2696.0


In [10]:
# Most competitive Job Listing on Indeed
df.loc[df['maskedEmployerName'] == 'Company824633721742']

,date,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,candStatusReviewedCount,supervisingJob,...,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash,totalClicks,totalApplies,jobsListedByEmp
85589,2017-05-02,Company824633721742,20130418,Direct Employer,0,20,20170412,393,116,0,...,1-49,passenger service agent,Transportation/Logistics,customer,PA,Philadelphia,Company82463372174252,17413,4312,1


In [11]:
len(df)

236028

In [12]:
null_data = df[df.isnull().any(axis=1)]

In [13]:
len(null_data)

667

In [31]:
df = df.dropna(axis=1, how='any')

In [32]:
len(no_null)

236028

In [33]:
len(df)

236028

# Step 3 - One Hot Encoding

In [17]:
import numpy as np
import pandas as pd

In [18]:
df = pd.read_csv('datafest_cleanv2.csv')

In [19]:
df.head()

,date,maskedEmployerName,employerAccountDateCreated,advtype,avgoverallcompanyrating,jobagedays,jobdatecreated,descriptionLengthChars,candStatusReviewedCount,supervisingJob,...,employeecount,normtitle,employerIndustry,normtitlecategory,admin1,city,jobHash,totalClicks,totalApplies,jobsListedByEmp
0,2016-12-01,Company1288490188873,20160211,Direct Employer,0,16,20161115,2116,2,0,...,150-249,solar installer,Technology/Information,install,DC,Washington,Company1288490188873305,27,4,11
1,2016-12-01,Company1288490188873,20160211,Direct Employer,0,0,20161201,3073,6,0,...,1-49,senior special events manager,Technology/Information,management,MD,Columbia,Company1288490188873410,43,8,11
2,2016-12-03,Company1288490188873,20160211,Direct Employer,0,0,20161203,1882,1,0,...,1-49,receptionist/clerk,Technology/Information,admin,FL,ChampionsGate,Company1288490188873431,19,2,11
3,2016-12-19,Company1288490188873,20160211,Direct Employer,0,12,20161207,1239,1,0,...,1-49,junior support engineer,Technology/Information,techhelp,IL,Farina,Company1288490188873469,174,27,11
4,2017-02-04,Company1288490188873,20160211,Direct Employer,0,1,20170203,2116,0,0,...,1-49,financial analyst,Technology/Information,accounting,VA,Arlington,Company1288490188873725,50,7,11


In [20]:
# Drop Dates
df = df.drop(['date','employerAccountDateCreated','jobdatecreated','advDateCreatedDate'], axis=1)
# Drop Strings
df = df.drop(['maskedEmployerName','normtitle','normtitlecategory','city'], axis=1)
# Drop Irrelevant
df = df.drop(['yearWeekOfYear'], axis=1)

In [21]:
# print(len(df['maskedEmployerName'].unique()))
# print(len(df['advtype'].unique()))
# print(len(df['employeecount'].unique()))
# print(len(df['normtitle'].unique()))
# print(len(df['normtitlecategory'].unique()))
# print(len(df['city'].unique()))

In [22]:
# df = df[['employerIndustry','admin1','advtype','employeecount',]]

In [23]:
df.head()

,advtype,avgoverallcompanyrating,jobagedays,descriptionLengthChars,candStatusReviewedCount,supervisingJob,licenseRequiredJob,noEducationRequirementsJob,highSchoolEducationRequirementsJob,higherEducationRequirementsJob,employeecount,employerIndustry,admin1,jobHash,totalClicks,totalApplies,jobsListedByEmp
0,Direct Employer,0,16,2116,2,0,0,0,0,0,150-249,Technology/Information,DC,Company1288490188873305,27,4,11
1,Direct Employer,0,0,3073,6,0,0,0,0,0,1-49,Technology/Information,MD,Company1288490188873410,43,8,11
2,Direct Employer,0,0,1882,1,0,0,0,0,0,1-49,Technology/Information,FL,Company1288490188873431,19,2,11
3,Direct Employer,0,12,1239,1,0,0,0,0,0,1-49,Technology/Information,IL,Company1288490188873469,174,27,11
4,Direct Employer,0,1,2116,0,0,0,0,0,1,1-49,Technology/Information,VA,Company1288490188873725,50,7,11


In [24]:
print(len(df['avgoverallcompanyrating'].unique()))

42


In [25]:
# Get one hot encoding of columns B
one_hot = pd.get_dummies(df[['employerIndustry','admin1','advtype','employeecount']])

In [26]:
# Drop column B as it is now encoded
df = df.drop(['employerIndustry','admin1','advtype','employeecount'], axis=1)

In [27]:
# Join the encoded df
df = df.join(one_hot)

In [28]:
columns = list(df.columns)
for col in columns:
    print(col)

avgoverallcompanyrating
jobagedays
descriptionLengthChars
candStatusReviewedCount
supervisingJob
licenseRequiredJob
noEducationRequirementsJob
highSchoolEducationRequirementsJob
higherEducationRequirementsJob
jobHash
totalClicks
totalApplies
jobsListedByEmp
employerIndustry_Accounting/Consulting
employerIndustry_Advertising/Marketing
employerIndustry_Agriculture
employerIndustry_Arts/Entertainment/Recreation
employerIndustry_Automotive
employerIndustry_Beauty/Wellness
employerIndustry_Construction/Engineering
employerIndustry_Defense/Security
employerIndustry_Education
employerIndustry_Energy/Utilities
employerIndustry_Financial Services/Banking
employerIndustry_Food/Beverage Services
employerIndustry_Government/Military
employerIndustry_Health Care/Medical
employerIndustry_Hospitality/Travel
employerIndustry_Insurance
employerIndustry_Job Board (General)
employerIndustry_Legal
employerIndustry_Manufacturing
employerIndustry_Media/Newspaper
employerIndustry_Non-Profit/Association
emplo

In [29]:
len(df)

235361

In [30]:
len(df.columns)

110

In [31]:
df.dtypes

avgoverallcompanyrating                            int64
jobagedays                                         int64
descriptionLengthChars                             int64
candStatusReviewedCount                            int64
supervisingJob                                     int64
licenseRequiredJob                                 int64
noEducationRequirementsJob                         int64
highSchoolEducationRequirementsJob                 int64
higherEducationRequirementsJob                     int64
jobHash                                           object
totalClicks                                        int64
totalApplies                                       int64
jobsListedByEmp                                    int64
employerIndustry_Accounting/Consulting             uint8
employerIndustry_Advertising/Marketing             uint8
employerIndustry_Agriculture                       uint8
employerIndustry_Arts/Entertainment/Recreation     uint8
employerIndustry_Automotive    

## Randomize

In [32]:
df.set_index('jobHash')

,avgoverallcompanyrating,jobagedays,descriptionLengthChars,candStatusReviewedCount,supervisingJob,licenseRequiredJob,noEducationRequirementsJob,highSchoolEducationRequirementsJob,higherEducationRequirementsJob,totalClicks,...,advtype_Other,advtype_Staffing Agency,advtype_none,employeecount_1-49,employeecount_1000+,employeecount_150-249,employeecount_250-499,employeecount_50-149,employeecount_500-749,employeecount_750-999
jobHash,,,,,,,,,,,,,,,,,,,,,
Company1288490188873305,0,16,2116,2,0,0,0,0,0,27,...,0,0,0,0,0,1,0,0,0,0
Company1288490188873410,0,0,3073,6,0,0,0,0,0,43,...,0,0,0,1,0,0,0,0,0,0
Company1288490188873431,0,0,1882,1,0,0,0,0,0,19,...,0,0,0,1,0,0,0,0,0,0
Company1288490188873469,0,12,1239,1,0,0,0,0,0,174,...,0,0,0,1,0,0,0,0,0,0
Company1288490188873725,0,1,2116,0,0,0,0,0,1,50,...,0,0,0,1,0,0,0,0,0,0
Company1288490188873756,0,1,1863,0,0,0,0,0,1,86,...,0,0,0,1,0,0,0,0,0,0
Company1288490188873902,0,3,1855,1,0,0,0,0,0,31,...,0,0,0,1,0,0,0,0,0,0
Company1288490188873893,0,6,1912,0,0,0,1,0,0,69,...,0,0,0,1,0,0,0,0,0,0
Company12884901888731030,0,9,2156,0,0,0,0,0,0,170,...,0,0,0,1,0,0,0,0,0,0


In [33]:
df = df.sample(frac=1).reset_index(drop=True)
df.set_index('jobHash')
df.head(10)

,avgoverallcompanyrating,jobagedays,descriptionLengthChars,candStatusReviewedCount,supervisingJob,licenseRequiredJob,noEducationRequirementsJob,highSchoolEducationRequirementsJob,higherEducationRequirementsJob,jobHash,...,advtype_Other,advtype_Staffing Agency,advtype_none,employeecount_1-49,employeecount_1000+,employeecount_150-249,employeecount_250-499,employeecount_50-149,employeecount_500-749,employeecount_750-999
0,0,14,557,3,0,0,0,0,0,Company936302870699125,...,0,0,1,0,0,0,0,0,0,1
1,0,61,378,13,0,0,0,0,0,Company2834678422371,...,0,0,0,0,0,0,0,0,1,0
2,0,98,1622,1,0,0,0,0,0,Company11166915136534,...,0,0,0,0,0,1,0,0,0,0
3,0,28,1241,2,0,0,0,0,0,Company16664473304361,...,0,0,1,0,0,1,0,0,0,0
4,39,31,2084,2,0,0,0,0,0,Company343597384021085,...,0,0,0,1,0,0,0,0,0,0
5,45,6,2949,26,0,0,0,0,0,Company1391569404145166,...,0,0,0,0,0,0,0,0,0,1
6,0,11,1567,5,0,0,0,0,0,Company197568496474328,...,0,0,0,1,0,0,0,0,0,0
7,0,13,2072,1,0,0,0,0,0,Company94489280568506,...,0,0,0,0,0,0,0,0,1,0
8,0,35,1216,0,0,1,1,0,0,Company33500744994595,...,0,0,0,0,0,0,0,1,0,0
9,40,1,851,2,0,0,0,0,0,Company309237646052152,...,0,1,0,0,0,0,0,1,0,0


In [34]:
df.set_index('jobHash')

,avgoverallcompanyrating,jobagedays,descriptionLengthChars,candStatusReviewedCount,supervisingJob,licenseRequiredJob,noEducationRequirementsJob,highSchoolEducationRequirementsJob,higherEducationRequirementsJob,totalClicks,...,advtype_Other,advtype_Staffing Agency,advtype_none,employeecount_1-49,employeecount_1000+,employeecount_150-249,employeecount_250-499,employeecount_50-149,employeecount_500-749,employeecount_750-999
jobHash,,,,,,,,,,,,,,,,,,,,,
Company936302870699125,0,14,557,3,0,0,0,0,0,384,...,0,0,1,0,0,0,0,0,0,1
Company2834678422371,0,61,378,13,0,0,0,0,0,2529,...,0,0,0,0,0,0,0,0,1,0
Company11166915136534,0,98,1622,1,0,0,0,0,0,1300,...,0,0,0,0,0,1,0,0,0,0
Company16664473304361,0,28,1241,2,0,0,0,0,0,802,...,0,0,1,0,0,1,0,0,0,0
Company343597384021085,39,31,2084,2,0,0,0,0,0,820,...,0,0,0,1,0,0,0,0,0,0
Company1391569404145166,45,6,2949,26,0,0,0,0,0,1407,...,0,0,0,0,0,0,0,0,0,1
Company197568496474328,0,11,1567,5,0,0,0,0,0,280,...,0,0,0,1,0,0,0,0,0,0
Company94489280568506,0,13,2072,1,0,0,0,0,0,117,...,0,0,0,0,0,0,0,0,1,0
Company33500744994595,0,35,1216,0,0,1,1,0,0,69,...,0,0,0,0,0,0,0,1,0,0


## Engineer Target Variable

In [47]:
df['target'] = np.where(df['totalClicks']>=1000, 1, 0)

In [49]:
df = df.drop(['totalClicks'], axis=1)

In [50]:
df.head()

,avgoverallcompanyrating,jobagedays,descriptionLengthChars,candStatusReviewedCount,supervisingJob,licenseRequiredJob,noEducationRequirementsJob,highSchoolEducationRequirementsJob,higherEducationRequirementsJob,jobHash,...,advtype_Staffing Agency,advtype_none,employeecount_1-49,employeecount_1000+,employeecount_150-249,employeecount_250-499,employeecount_50-149,employeecount_500-749,employeecount_750-999,target
0,0,14,557,3,0,0,0,0,0,Company936302870699125,...,0,1,0,0,0,0,0,0,1,0
1,0,61,378,13,0,0,0,0,0,Company2834678422371,...,0,0,0,0,0,0,0,1,0,1
2,0,98,1622,1,0,0,0,0,0,Company11166915136534,...,0,0,0,0,1,0,0,0,0,1
3,0,28,1241,2,0,0,0,0,0,Company16664473304361,...,0,1,0,0,1,0,0,0,0,0
4,39,31,2084,2,0,0,0,0,0,Company343597384021085,...,0,0,1,0,0,0,0,0,0,0


## Split Train and Test Sets

In [51]:
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [52]:
len(train)

141216

In [53]:
len(test)

47073

In [54]:
len(validate)

47072

### Export Train, Test, Valid to CSV

In [55]:
train.to_csv('train.csv', index=False)

In [56]:
validate.to_csv('validate.csv', index=False)

In [57]:
test.to_csv('test.csv', index=False)

# Step 4 - Predictive Modeling

**Goal: Predict whether the job listing will get 1000 or more total clicks over the life of it's posting on Indeed.com**

In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [18]:
# set random seed
seed = 7
np.random.seed(seed)

In [19]:
dataframe = pd.read_csv('train.csv')

In [20]:
dataframe = dataframe.drop(['jobHash'], axis=1)

In [21]:
dataset = dataframe.values

In [22]:
X = dataset[:,0:108].astype(float)
Y = dataset[:,108]

In [23]:
len(X)

141216

In [24]:
X.shape

(141216, 108)

In [25]:
Y.shape

(141216,)

In [27]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

### Train Deep Neural Network using Keras with TensorFlow backend

**Two hidden layers using Adam optimizer and K-Fold Cross-validation**

In [28]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(108, input_dim=108, kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, input_dim=108, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [29]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=20, batch_size=10, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 98.16% (0.64%)


In [30]:
results

array([ 0.98775048,  0.98321767,  0.9803144 ,  0.97634896,  0.98569607,
        0.96947808,  0.98852772,  0.98654486,  0.98597833,  0.9721691 ])